# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob
from keras.utils import to_categorical

Using TensorFlow backend.


KeyboardInterrupt: 

# Load processed dataset
The data we load here are already shuffled and normalized.

In [ ]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

# Tunable parameters

In [ ]:
epochs = 5000
batch_size = 20
learning_rate = 0.0001
decay = 0.00001

# Load auto encoders

In [ ]:
autoencoder_path = 'results/25-stacked-autoencoder-01/model/2018-03-07-04-14-10/weights.hdf5'
weights = load_model(autoencoder_path)
weights.summary()

# Set up constants for weight transfer

In [ ]:
autoencoder_layer_1_units = 10
autoencoder_layer_2_units = 8
autoencoder_layer_1_name = 'dense_4'
autoencoder_layer_2_name = 'dense_5'
autoencoder_layer_3_name = 'dense_6'
autoencoder_layer_1_dropout = 0.3/2
autoencoder_layer_2_dropout = 0.3

# Model for classification

In [ ]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/26-fine-tune-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"

def create_model(layers, dropout):
    global date
    global autoencoder_path
    x = Input(shape=(x_train.shape[1],))
    
    y = Dense(units = autoencoder_layer_1_units, activation='relu')(x)
    y = Dropout(autoencoder_layer_1_dropout)(y)
    
    y = Dense(units = autoencoder_layer_2_units, activation='relu')(y)
    y = Dropout(autoencoder_layer_2_dropout)(y)
    
    y = Dense(units = x_train.shape[1], activation='relu')(y)
    y = Dropout(dropout/3)(y)

    y = Dense(units = layers['layer_1'], activation='relu')(y)
    y = Dropout(dropout/2)(y)
    
    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    if (layers['layer_3'] > 0):
        y = Dense(units = layers['layer_3'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    y = Dense(units = y_train.shape[1], activation='softmax')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 4': layers['layer_1'],
        'Layer 5': layers['layer_2'],
        'Layer 6': layers['layer_3'],
        'Dropout 4': dropout/3,
        'Dropout 5': dropout/2,
        'Dropout 6': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Batch size': batch_size,
        'Autoencoder': autoencoder_path,
        'Epochs': epochs,
        'Fine tune': True
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    # Set layer weights to pre trained autoencoder weights
    model.layers[1].set_weights(weights.get_layer(autoencoder_layer_1_name).get_weights())
    model.layers[3].set_weights(weights.get_layer(autoencoder_layer_2_name).get_weights())
    model.layers[5].set_weights(weights.get_layer(autoencoder_layer_3_name).get_weights())
    
    return model

# Prepare callbacks

In [ ]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, verbose=0, **kwargs)

# Train

In [ ]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [
    {'layer_1': 32, 'layer_2': 16, 'layer_3': 8},
    {'layer_1': 24, 'layer_2': 16, 'layer_3': 8},
    {'layer_1': 24, 'layer_2': 16, 'layer_3': 0},
    {'layer_1': 64, 'layer_2': 32, 'layer_3': 16},
    {'layer_1': 64, 'layer_2': 16, 'layer_3': 0},
    {'layer_1': 16, 'layer_2': 8, 'layer_3': 4},
    {'layer_1': 8, 'layer_2': 4, 'layer_3': 0},
    {'layer_1': 16, 'layer_2': 0, 'layer_3': 0},
    {'layer_1': 8, 'layer_2': 12, 'layer_3': 16},
    {'layer_1': 16, 'layer_2': 8, 'layer_3': 16}
]
param_grid = dict(
    layers=layers,
    dropout=[0.3, 0.5]
)
    
grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, y_test)))
grid_result = grid.fit(x_train, y_train)